In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import numpy as np
import matplotlib.pyplot as plt

# 1. Đọc dữ liệu từ file CSV
data = pd.read_csv('creditcard.csv')

# 2. Kiểm tra phân phối nhãn
print(data['Class'].value_counts())

# 3. Chuẩn hóa các cột Amount và Time
scaler = StandardScaler()

data['Amount_scaled'] = scaler.fit_transform(data[['Amount']])
data['Time_scaled'] = scaler.fit_transform(data[['Time']])

# 4. Chọn tập huấn luyện chỉ chứa dữ liệu bình thường
train_data_normal = data[data['Class'] == 0].sample(n=10000, random_state=42)

# 5. Chọn tập kiểm tra gồm dữ liệu bình thường và bất thường
test_data_normal = data[data['Class'] == 0].sample(n=800, random_state=42)
test_data_fraud = data[data['Class'] == 1].sample(n=400, random_state=42)

# 6. Kết hợp tập kiểm tra
test_data = pd.concat([test_data_normal, test_data_fraud])
test_data = test_data.sample(frac=1, random_state=42).reset_index(drop=True)  # Shuffle test set

# 7. Tạo X_train và X_test, bỏ cột Class, Amount và Time gốc
X_train = train_data_normal.drop(columns=['Class', 'Amount', 'Time'])
X_test = test_data.drop(columns=['Class', 'Amount', 'Time'])

# 8. Gán lại nhãn cho y_test
y_test = test_data['Class']

# 9. Thông tin kích thước tập dữ liệu
print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")

# (Optional) Kiểm tra phân phối dữ liệu kiểm tra
print(y_test.value_counts())


Class
0    284315
1       492
Name: count, dtype: int64
Train size: 10000, Test size: 1200
Class
0    800
1    400
Name: count, dtype: int64


In [2]:
# Bước 5: Khởi tạo Isolation Forest với tham số điều chỉnh
iso_forest = IsolationForest(n_estimators=200, contamination=0.002, random_state=42)

# Bước 6: Huấn luyện mô hình trên dữ liệu huấn luyện
iso_forest.fit(X_train)

# Bước 7: Dự đoán trên tập kiểm tra
y_pred_test = iso_forest.predict(X_test)

# Isolation Forest trả về -1 cho bất thường và 1 cho bình thường, do đó cần chuyển đổi nhãn
y_pred_test = np.where(y_pred_test == -1, 1, 0)  # 1 là bất thường (gian lận), 0 là bình thường

# Bước 8: Tính toán điểm bất thường (anomaly scores)
anomaly_scores = iso_forest.decision_function(X_test)


In [3]:
# Bước 9: Điều chỉnh ngưỡng phát hiện bất thường (chọn ngưỡng dựa trên phân vị thứ 5)
threshold = np.percentile(anomaly_scores, 5)  # 5% điểm thấp nhất có thể là bất thường

# Dự đoán lại dựa trên ngưỡng mới
y_pred_test_adjusted = (anomaly_scores < threshold).astype(int)

In [4]:
# ==== 10. Huấn luyện IsolationForest trên dữ liệu normal ====
# Lưu ý: score_samples càng CAO thì càng "bình thường".
# Ta sẽ đảo dấu để có 'anom_score' càng CAO càng "bất thường".
iso = IsolationForest(
    n_estimators=200,
    max_samples='auto',
    contamination='auto',   # không dùng predict() nên tham số này ít ảnh hưởng
    random_state=42,
    n_jobs=-1
)
iso.fit(X_train)

# ==== 11. Tính điểm bất thường liên tục ====
# score_samples: cao = normal, thấp = bất thường → đảo dấu cho dễ hiểu
raw_scores = iso.score_samples(X_test)   # hình dạng (n_samples,)
anom_score = -raw_scores                 # cao = bất thường

# (tuỳ chọn) xem min/max để hiểu phân bố
print(f"anom_score range: {anom_score.min():.4f} ~ {anom_score.max():.4f}")

# ==== 12. Điều chỉnh ngưỡng (ví dụ: lấy top 5% điểm cao nhất là bất thường) ====
top_pct = 5  # bạn đổi 5 -> 10/15... để tăng Recall
threshold = np.percentile(anom_score, 100 - top_pct)  # ngưỡng tại bách phân vị 95

# Dự đoán: anom_score >= threshold => 1 (bất thường), ngược lại 0 (bình thường)
y_pred_test_adjusted = (anom_score >= threshold).astype(int)

# ==== 13. Đánh giá ====
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test_adjusted))
print("Classification Report:\n", classification_report(y_test, y_pred_test_adjusted, digits=2))

# ROC AUC nên tính bằng ĐIỂM liên tục (anom_score), không phải nhãn 0/1
print("ROC AUC Score (using scores):", roc_auc_score(y_test, anom_score))



anom_score range: 0.3539 ~ 0.7260
Confusion Matrix:
 [[800   0]
 [340  60]]
Classification Report:
               precision    recall  f1-score   support

           0       0.70      1.00      0.82       800
           1       1.00      0.15      0.26       400

    accuracy                           0.72      1200
   macro avg       0.85      0.57      0.54      1200
weighted avg       0.80      0.72      0.64      1200

ROC AUC Score (using scores): 0.9544968749999999
